In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [2]:
data1 = pd.read_csv('/work/Daily_Cuaca_Kebumen.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/work/Daily_Cuaca_Kebumen.csv'

In [3]:
data2 = pd.read_csv('/work/Kebumen 2023-01-01 to 2024-12-31.csv')

In [4]:
data1.tail(10)

In [5]:
data2.tail(10)

In [6]:
data3 = pd.concat([data1, data2], ignore_index=True)

data3

In [7]:
# Menyimpan DataFrame ke file Excel
data3.to_excel('data 1978 - 2024.xlsx', index=False)

In [8]:
data3.describe()

In [9]:
msno.matrix(data3)

In [10]:
data3.info()

In [11]:
data3

In [12]:
data3['datetime'] = pd.to_datetime(data3['datetime'])

In [13]:
data3.isnull().sum()

# Penyaringan Data

In [14]:
# Pilih kolom yang diinginkan
selected_columns = [
    'datetime',
    'tempmax', 
    'tempmin', 
    'temp',  
    'humidity', 
    'precip', 
    'sealevelpressure',
    'windspeed',
    'winddir',
    'precipprob'
]

# Membuat DataFrame baru hanya dengan kolom yang dipilih
data_filtered = data3[selected_columns]

# Menampilkan DataFrame hasil seleksi
print(data_filtered.head())

In [15]:
data_filtered

In [16]:
data_filtered.plot(
    y=[
    'tempmax', 
    'tempmin', 
    'temp',  
    'humidity', 
    'precip', 
    'sealevelpressure',
    'windspeed',
    'winddir',
    'precipprob'
    ],
    subplots=True, figsize=(10, 15))
plt.show()

for column in ['tempmax', 'tempmin', 'temp', 'humidity', 'precip', 'sealevelpressure', 'windspeed', 'winddir', 'precipprob']:
    sns.histplot(data_filtered[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

# Pembersihan Data - Bagian Null Value

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

# Daftar kolom yang akan digunakan
columns_to_clean = [
    'tempmax', 
    'tempmin', 
    'temp',  
    'humidity', 
    'precip', 
    'sealevelpressure',
    'windspeed',
    'winddir'
]

# Inisialisasi KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")  # Menggunakan 5 tetangga terdekat

# Mengganti NaN dengan KNN Imputer dan menyimpan hasil ke DataFrame baru
data3_cleaned = data_filtered.copy()
data3_cleaned[columns_to_clean] = knn_imputer.fit_transform(data_filtered[columns_to_clean])

# Visualisasi data setelah menggantikan NaN
data3_cleaned.plot(
    y=columns_to_clean,
    subplots=True,
    figsize=(10, 15),
    title='Data Setelah Imputasi dengan KNN'
)
plt.show()


In [18]:
# Mengganti nilai NaN di kolom 'precipprob' dengan 0
data3_cleaned['precipprob'] = data3_cleaned['precipprob'].fillna(0)

In [19]:
data3_cleaned.isnull().sum()

In [20]:
# Daftar kolom yang akan divisualisasikan
columns = ['tempmax', 'tempmin', 'temp',  'humidity', 'precip', 
           'sealevelpressure', 'windspeed', 'winddir', 'precipprob']

# Loop untuk membuat histogram dan boxplot
for column in columns:
    # Histogram
    plt.figure(figsize=(10, 5))
    sns.histplot(data3_cleaned[column], kde=True, color='blue')
    plt.title(f'Distribution of {column} (Histogram)')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()
    
    # Boxplot
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=data3_cleaned[column], color='orange')
    plt.title(f'Distribution of {column} (Boxplot)')
    plt.xlabel(column)
    plt.show()

# Pembersihan Data - Bagian Outliers 

In [21]:
import numpy as np
import pandas as pd

# Salin data asli
data_fix = data3_cleaned.copy()

# Daftar kolom yang akan dibersihkan dari outlier
columns_to_clean = [
    'tempmax', 
    'tempmin', 
    'temp', 
    'humidity',                 
    'sealevelpressure', 
    'windspeed'
]

# Fungsi untuk mengganti outlier menggunakan IQR
def replace_outliers_with_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Kuartil 1
    Q3 = df[column].quantile(0.75)  # Kuartil 3
    IQR = Q3 - Q1  # Interquartile Range
    lower_bound = Q1 - 1.5 * IQR  # Batas bawah
    upper_bound = Q3 + 1.5 * IQR  # Batas atas

    # Identifikasi outliers
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)

    # Ganti outliers dengan median kolom
    median_value = df[column].median()
    df.loc[outliers, column] = median_value

# Terapkan IQR pada setiap kolom dalam daftar
for column in columns_to_clean:
    replace_outliers_with_iqr(data_fix, column)

# Tampilkan data yang telah dibersihkan
print("Data setelah menggantikan outliers menggunakan IQR:")
print(data_fix.head(10))


In [22]:
# Visualisasi data setelah menggantikan NaN
data_fix.plot(
    subplots=True,
    figsize=(10, 15),
    title='Data setelah pembersihan Outlier'
)
plt.show()

In [23]:
# Daftar kolom yang akan divisualisasikan
columns = ['tempmax', 'tempmin', 'temp', 'humidity', 'precip', 
           'sealevelpressure', 'windspeed', 'winddir', 'precipprob']

# Loop untuk membuat histogram dan boxplot
for column in columns:
    # Histogram
    plt.figure(figsize=(10, 5))
    sns.histplot(data_fix[column], kde=True, color='blue')
    plt.title(f'Distribution of {column} (Histogram)')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()
    
    # Boxplot
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=data_fix[column], color='orange')
    plt.title(f'Distribution of {column} (Boxplot)')
    plt.xlabel(column)
    plt.show()

In [24]:
# Memilih hanya kolom yang diinginkan
selected_columns = [
    'tempmax', 
    'tempmin', 
    'temp',  
    'humidity', 
    'precip', 
    'sealevelpressure',
    'windspeed'

]

# Menghitung matriks korelasi hanya untuk kolom yang dipilih
corrmat = data_fix[selected_columns].corr()

# Menentukan palet warna untuk heatmap
cmap = sns.color_palette("RdBu", as_cmap=True)

# Membuat heatmap dengan ukuran yang sesuai
plt.figure(figsize=(12,12))
sns.heatmap(corrmat, cmap=cmap, annot=True, square=True)
plt.title('Korelasi Heatmap antar Variabel')
plt.show()

# Data Preproses - Membuat Target (Y)

In [25]:
# Rename the column 'precipprob' to 'precipToday'
data_fix.rename(columns={'precipprob': 'precipToday'}, inplace=True)

In [26]:
# Menambahkan kolom klasifikasi precip
def classify_precip(value):
    if 0 <= value <=0.1:
        return 'No Rain'
    elif 0.1 <= value <= 19.9:
        return 'Light'
    elif 20.0 <= value <= 49.9:
        return 'Medium'
    elif 50.0 <= value <= 99.9:
        return 'Heavy'
    elif value >= 100.0:
        return 'Extrem'
    else:
        return 'Unknown'  # Untuk nilai yang tidak masuk dalam range

# Terapkan klasifikasi ke kolom precip
data_fix['precip_class_Today'] = data_fix['precip'].apply(classify_precip)

### Penggeseran Kolom biner hujan

In [27]:
# Geser kolom
data_fix['precipTomorrow'] = data_fix['precipToday'].shift(-1)

# Isi nilai NaN dengan median
fill_value = data_fix['precipToday'].median()
data_fix['precipTomorrow']= data_fix['precipTomorrow'].fillna(fill_value)

In [28]:
# Geser kolom
data_fix['precip_class_Tomorrow'] = data_fix['precip_class_Today'].shift(-1)

data_fix['precip_class_Tomorrow']= data_fix['precip_class_Tomorrow'].fillna('No Rain')

In [29]:
sns.countplot(x=data_fix["precip_class_Today"])

In [30]:
sns.countplot(x= data_fix["precipToday"])

In [31]:
sns.countplot(x=data_fix["precip_class_Tomorrow"])

In [32]:
sns.countplot(x= data_fix["precipTomorrow"])

In [33]:

# Membuat pairplot dengan palet warna dan penyesuaian lainnya
pairplot = sns.pairplot(
    data=data_fix,
    vars=[
        'tempmax', 
        'tempmin', 
        'temp', 
        'humidity', 
        'precip',
        'sealevelpressure',
        'windspeed'
    ],
    hue='precip_class_Tomorrow',  # Warna berdasarkan kategori precipprob
    diag_kind="kde",  # Kernel Density Estimation untuk distribusi diagonal
    height=2,  # Ukuran setiap plot
    aspect=1.2  # Rasio panjang-lebar plot
)

# Menambahkan judul untuk keseluruhan grafik
pairplot.fig.suptitle(
    "Pairplot of Weather Features by Precipitation Classification",
    y=1.02,  # Penyesuaian jarak dari grafik ke judul
    fontsize=16
)

# Menampilkan grafik
plt.savefig("Pairplot_Features.jpg")
plt.show()

In [34]:
data_fix

In [35]:
data_fix.info()

In [36]:
data_fix.describe()

In [37]:
data_fix.isna().sum()

In [38]:
# Library Dasar
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Model dan Algoritma
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Data Processing dan Evaluasi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

# Membagi variabel independen dan target
X = data_fix[[
    'temp', 
    'tempmax',
    'tempmin',
    'humidity',
    'precip',
    'sealevelpressure',
    'windspeed'
]]  # Variabel independen
y = data_fix['precip_class_Tomorrow']  # Target (multiclass classification)

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Normalisasi fitur numerik
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Binarisasi target untuk ROC AUC multikelas
classes = y.unique()  # Daftar kelas
y_train_bin = label_binarize(y_train, classes=classes)
y_test_bin = label_binarize(y_test, classes=classes)
n_classes = y_train_bin.shape[1]

# Daftar model klasifikasi
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=100),
    'Random Forest': RandomForestClassifier(random_state=100),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Support Vector Machine': SVC(probability=True, random_state=100),
    'Neural Networks': MLPClassifier(random_state=100, max_iter=1000)
}

# Evaluasi model dan penyimpanan hasil
results = []

# Plot ROC Curve untuk setiap model secara terpisah
for idx, (name, model) in enumerate(models.items()):
    # Melatih model
    model.fit(X_train_scaled, y_train)
    
    # Prediksi
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)  # Probabilitas untuk semua kelas

    # Evaluasi performa dasar
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision (macro avg)': report['macro avg']['precision'],
        'Recall (macro avg)': report['macro avg']['recall'],
        'F1-Score (macro avg)': report['macro avg']['f1-score']
    })
    
    # Menampilkan Confusion Matrix
    cm = confusion_matrix(y_test, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f'Confusion Matrix for {name}')
    plt.show()
    
    # Plot ROC Curve untuk tiap kelas dalam model
    plt.figure(figsize=(8, 6))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(
            fpr, 
            tpr, 
            linestyle='-', 
            label=f'Class {classes[i]} (AUC = {roc_auc:.2f})')
    
    # Menambahkan garis diagonal sebagai referensi untuk prediksi acak
    plt.plot([0, 1], [0, 1], color='black', linestyle='--', label='Random Prediction')
    
    # Pengaturan grafik ROC untuk model
    plt.title(f'ROC Curve for {name}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right', fontsize='small')
    plt.grid(True)
    plt.show()

# Menampilkan tabel hasil evaluasi
results_df = pd.DataFrame(results)
print("Hasil Evaluasi Model:")
print(results_df)


In [39]:
data_fix.head(10)

In [40]:
import pandas as pd
import matplotlib.pyplot as plt

# Konversi kolom 'datetime' menjadi format datetime jika belum dilakukan
data_fix['datetime'] = pd.to_datetime(data_fix['datetime'])

# Set 'datetime' sebagai index untuk analisis waktu
data_fix.set_index('datetime', inplace=True)


In [41]:
# Menambahkan kolom satuan waktu dari variabel datetime
data_fix['year'] = data_fix.index.year
data_fix['month'] = data_fix.index.month
data_fix['day'] = data_fix.index.day

# Agregasi curah hujan berdasarkan satuan waktu tertentu
# Curah hujan harian
daily_rainfall = data_fix['precip'].resample('D').sum()

# Curah hujan bulanan
monthly_rainfall = data_fix['precip'].resample('M').sum()

# Curah hujan tahunan
yearly_rainfall = data_fix['precip'].resample('Y').sum()

# Curah hujan musiman
seasonal_rainfall = data_fix['precip'].resample('Q').sum()

# Menghitung rata-rata curah hujan per bulan (menggabungkan semua tahun)
average_monthly_rainfall = data_fix.groupby('month')['precip'].mean()

# Menghitung rata-rata curah hujan per tahun
average_yearly_rainfall = data_fix.groupby('year')['precip'].mean()

# Visualisasi Curah Hujan Bulanan
plt.figure(figsize=(10, 6))
monthly_rainfall.plot(kind='line', color='skyblue', alpha=0.7)
plt.title("Curah Hujan Bulanan")
plt.xlabel("Bulan")
plt.ylabel("Curah Hujan (mm)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Visualisasi Curah Hujan Tahunan
plt.figure(figsize=(10, 6))
yearly_rainfall.plot(kind='bar', color='orange', alpha=0.7)
plt.title("Curah Hujan Tahunan")
plt.xlabel("Tahun")
plt.ylabel("Curah Hujan (mm)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Visualisasi Curah Hujan Musiman
plt.figure(figsize=(10, 6))
seasonal_rainfall.plot(kind='bar', color='purple', alpha=0.7)
plt.title("Curah Hujan Musiman")
plt.xlabel("Musim")
plt.ylabel("Curah Hujan (mm)")
plt.grid()
plt.tight_layout()
plt.show()

# Visualisasi rata-rata curah hujan per bulan
plt.figure(figsize=(10, 6))
average_monthly_rainfall.plot(kind='bar', color='dodgerblue', alpha=0.8)
plt.title("Rata-rata Curah Hujan Bulanan")
plt.xlabel("Bulan")
plt.ylabel("Rata-rata Curah Hujan (mm)")
plt.xticks(ticks=range(12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Visualisasi rata-rata curah hujan per tahun
plt.figure(figsize=(10, 6))
average_yearly_rainfall.plot(kind='bar', color='green', alpha=0.8)
plt.title("Rata-rata Curah Hujan Tahunan")
plt.xlabel("Tahun")
plt.ylabel("Rata-rata Curah Hujan (mm)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>